# ANALYTICS FOR UNSTRUCTURED DATA - FINAL PROJECT

# COHORT 2 GROUP 9
## April Kim, Bruce Kim, Monica Martinez, Boran Sheu, Pengwei Wang, Ziyu Wang

# Web scraping 

reference: https://www.linkedin.com/pulse/how-scrape-app-store-reviews-4-simple-steps-using-python-kundi/

*sometimes the API may have the following error:*

`ERROR:Base:Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url:/v1/catalog/us/apps/1017492454/reviews?l=en-GB&offset=0&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 429 error responses'))`

*restart the kernel and rerun*

In [ ]:
#!pip install app_store_scraper
#from app_store_scraper import AppStore

In [2]:
# review_per_app = 500

# apps = ['spotify-music-and-podcasts', 
#         'apple-music', 
#         'soundcloud-discover-new-music', 
#         'youtube-music', 
#         'amazon-music-songs-podcasts', 
#         'tidal-music']

# ids = ['324684580', 
#        '1108187390', 
#        '336353151', 
#        '1017492454', 
#        '510855668', 
#        '913943275']

In [3]:
# spotify = AppStore(country='us', app_name=apps[0], app_id = ids[0])
# spotify.review(how_many=review_per_app)

# apple = AppStore(country='us', app_name=apps[1], app_id = ids[1])
# apple.review(how_many=review_per_app)

# soundcloud = AppStore(country='us', app_name=apps[2], app_id = ids[2])
# soundcloud.review(how_many=review_per_app)

# youtube = AppStore(country='us', app_name=apps[3], app_id = ids[3])
# youtube.review(how_many=review_per_app)

# amazon = AppStore(country='us', app_name=apps[4], app_id = ids[4])
# amazon.review(how_many=review_per_app)

# tidal = AppStore(country='us', app_name=apps[5], app_id = ids[5])
# tidal.review(how_many=review_per_app)

In [9]:
# df0 = pd.DataFrame(np.array(spotify.reviews), columns=['review'])
# df_spot = df0.join(pd.DataFrame(df0.pop('review').tolist()))
# df_spot['appName'] = 'Spotify'

# df1 = pd.DataFrame(np.array(apple.reviews), columns=['review'])
# df_app = df1.join(pd.DataFrame(df1.pop('review').tolist()))
# df_app['appName'] = 'Apple Music'

# df2 = pd.DataFrame(np.array(soundcloud.reviews), columns=['review'])
# df_soundcloud = df2.join(pd.DataFrame(df2.pop('review').tolist()))
# df_soundcloud['appName'] = 'SoundCloud'

# df3 = pd.DataFrame(np.array(youtube.reviews), columns=['review'])
# df_youtube = df3.join(pd.DataFrame(df3.pop('review').tolist()))
# df_youtube['appName'] = 'YouTube Music'

# df4 = pd.DataFrame(np.array(amazon.reviews), columns=['review'])
# df_amazon = df4.join(pd.DataFrame(df4.pop('review').tolist()))
# df_amazon['appName'] = 'Amazon Music'

# df5 = pd.DataFrame(np.array(tidal.reviews), columns=['review'])
# df_tidal = df5.join(pd.DataFrame(df5.pop('review').tolist()))
# df_tidal['appName'] = 'Tidal'

In [10]:
# df = pd.concat([df_spot, df_app, df_soundcloud, df_youtube, df_amazon, df_tidal], ignore_index=True)
# df = df[['userName', 'appName', 'rating', 'title', 'review']]
# df.head()

In [11]:
# df.to_excel('music_reviews.xlsx', index=False)

# Analysis

In [1]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")

In [65]:
df = pd.read_excel('music_reviews.xlsx')

## Data Preprocessing 

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import sent_tokenize, word_tokenize
from string import punctuation
from collections import Counter
import re

all_words = []

def clean_review(review):
    lemmatizer = WordNetLemmatizer()
    text = review.lower()
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…-]', '', text)
    text = re.sub('[%s]' % re.escape(punctuation), '', text)
    text = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    for word in text:
        if (punctuation.find(word.lower()) == -1) and (word.lower() not in stop_words):
            all_words.append(word.lower())        
    text = [lemmatizer.lemmatize(word) for word in text]
    return ' '.join(text)

df1 = df.copy()
df1['review'] = df1['review'].map(lambda x: clean_review(x))

# spacytextblob sentiment analysis

In [70]:
# !pip install spacy
# !pip3 install spacytextblob
# !python -m spacy download en
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import pandas as pd
import requests
from spacy.lang.en.examples import sentences 

In [71]:
import en_core_web_md
nlp = en_core_web_md.load()
nlp.add_pipe('spacytextblob')

In [80]:
review_sent_score = []
review_sent_label = []

In [81]:
for i in range(len(df1.axes[0])):
    doc = nlp(df1['review'][i])
    sentiment = doc._.blob.polarity
    sentiment = round(sentiment,2)

    if sentiment > 0:
        sent_label = "Positive"
    else:
        sent_label = "Negative"

    review_sent_label.append(sent_label)
    review_sent_score.append(sentiment)

In [82]:
len(review_sent_score)

3000

In [83]:
doc._.blob.sentiment_assessments.assessments

[(['sound'], 0.4, 0.4, None),
 (['great'], 0.8, 0.75, None),
 (['horrible'], -1.0, 1.0, None),
 (['only'], 0.0, 1.0, None),
 (['multiple'], 0.0, 0.0, None),
 (['only'], 0.0, 1.0, None),
 (['great'], 0.8, 0.75, None),
 (['multiple'], 0.0, 0.0, None),
 (['basic'], 0.0, 0.125, None),
 (['broken'], -0.4, 0.4, None)]

In [84]:
df_s = df1.copy()
df_s["Sentiment Score"] = review_sent_score
df_s["Sentiment Label"] = review_sent_label

df_s

,userName,appName,rating,title,review,Sentiment Score,Sentiment Label
0,V.143,Spotify,5,Best App,this app is by far the best music service ive ...,0.37,Positive
1,intense ligma,Spotify,5,Beat app but...,i personally love this app but the only part i...,0.13,Positive
2,GoldenShadow127,Spotify,3,It keeps stopping,a problem i frequently have is that when liste...,-0.22,Negative
3,strwvrryle,Spotify,4,"bluetooth/wired connection to car, library format",i always connect to my scion ia via bluetooth ...,-0.01,Negative
4,DeezNutsForPresident,Spotify,3,Issues with playing music,before i start i wanted to say that i wouldve ...,-0.02,Negative
...,...,...,...,...,...,...,...
2995,austiniodtx,Tidal,5,High quality streaming,ive been a tidal user for a couple of year now...,0.64,Positive
2996,NateShay71,Tidal,4,"Great streaming service, could be amazing though",tidal is a great high quality way to stream mu...,0.09,Positive
2997,No. Just no.,Tidal,2,From 4 to 2 stars…,even after the recent update im still receivin...,0.06,Positive
2998,knowledge360,Tidal,1,Terrible customer service,my tidal stopped working one day i couldnt log...,0.17,Positive


In [86]:
sentiment_sum = df_s.groupby('appName').agg({'Sentiment Score': ['mean', 'min', 'max']})
sentiment_sum.columns = ['sent_mean', 'sent_min', 'sent_max']
sentiment_sum.reset_index()
sentiment_sum.sort_values('sent_mean', ascending=False)

,sent_mean,sent_min,sent_max
appName,,,
Tidal,0.15516,-0.34,0.64
SoundCloud,0.13990,-0.31,0.51
Amazon Music,0.12600,-0.33,0.58
YouTube Music,0.11400,-0.90,0.59
Apple Music,0.11054,-0.78,0.80
Spotify,0.10550,-0.30,0.49


By conducting sentiment analysis using spaCy, we can determine that the app with the most positive sentiment on average is Tidal, with SoundCloud and Amazon Music, coming in behind Tidal.

# LDA and further sentiment analysis

## Document Term Matrix

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

#spotify = df1[df1['appName'] == 'Spotify']
cv = CountVectorizer(stop_words='english')
df_cv = cv.fit_transform(df1['review'])
dtm = pd.DataFrame(df_cv.toarray(), columns=cv.get_feature_names())
dtm.index = df1.index
dtm

,aaaaaaallllll,aaalll,ab,abandon,abbey,abcdefu,abd,ability,abit,able,...,zoe,zombie,zone,zoom,zztop,über,𝑡ℎ𝑎𝑡,𝗅𝗈𝗎𝖽𝗅𝗒,𝘮𝘦,𝙑𝙀𝙍𝙔
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Look at top occuring words for each review, add additional stop words

In [6]:
from gensim import matutils, models
import scipy.sparse

tdm = dtm.T
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)


#What are the top words for each review?
top_dict = {}
for review in tdm.columns:
    top = tdm[review].sort_values(ascending=False).head(10)
    top_dict[review]= list(zip(top.index, top.values))

top_dict

words = []
for review in tdm.columns:
    top = [word for (word, count) in top_dict[review]]
    for t in top:
        words.append(t)
        
#Look at how many reviews these top words appear in
Counter(words).most_common()

#Redundant words such as app, song, music does not give us information regarding features 
add_stop_words = [word for word, count in Counter(words).most_common() if count > 320 and (word != 'time' and word != 'playlist')]
add_stop_words

['song', 'app', 'music', 'like', 'play', 'just', 'wa', 'listen', 'dont', 'im']

## Re-vectorize Document Term Matrix 

In [7]:
from sklearn.feature_extraction import text 

stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

cv = CountVectorizer(stop_words = stop_words)
df_cv = cv.fit_transform(df1['review'])
dtm = pd.DataFrame(df_cv.toarray(), columns = cv.get_feature_names())

## Run LDA with different number of topics

In [8]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

tdm = dtm.T
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes

After running experiments, we decided to use the four-topic model

In [9]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics = 4, passes = 10)
lda.print_topics()

[(0,
  '0.020*"ad" + 0.016*"love" + 0.013*"soundcloud" + 0.011*"pay" + 0.011*"free" + 0.011*"really" + 0.008*"make" + 0.008*"want" + 0.008*"good" + 0.007*"know"'),
 (1,
  '0.014*"time" + 0.012*"ive" + 0.012*"service" + 0.011*"phone" + 0.009*"work" + 0.009*"issue" + 0.008*"fix" + 0.008*"problem" + 0.008*"apple" + 0.007*"month"'),
 (2,
  '0.055*"playlist" + 0.017*"want" + 0.016*"add" + 0.012*"time" + 0.011*"shuffle" + 0.010*"make" + 0.010*"artist" + 0.010*"love" + 0.009*"thing" + 0.009*"new"'),
 (3,
  '0.019*"tidal" + 0.016*"album" + 0.014*"artist" + 0.010*"track" + 0.010*"quality" + 0.009*"ha" + 0.008*"time" + 0.008*"streaming" + 0.008*"service" + 0.008*"apple"')]

## Below is an alternative method with more iterations --> Produces similar results

In [10]:
#!pip install lda
import lda

ntopics = 4
model = lda.LDA(n_topics=int(ntopics), n_iter=500, random_state=1)
model.fit(df_cv)

INFO:lda:n_documents: 3000
INFO:lda:vocab_size: 8758
INFO:lda:n_words: 138945
INFO:lda:n_topics: 4
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -1264238
INFO:lda:<10> log likelihood: -1122825
INFO:lda:<20> log likelihood: -1096807
INFO:lda:<30> log likelihood: -1086101
INFO:lda:<40> log likelihood: -1081156
INFO:lda:<50> log likelihood: -1077514
INFO:lda:<60> log likelihood: -1074496
INFO:lda:<70> log likelihood: -1073373
INFO:lda:<80> log likelihood: -1071819
INFO:lda:<90> log likelihood: -1070878
INFO:lda:<100> log likelihood: -1070103
INFO:lda:<110> log likelihood: -1069636
INFO:lda:<120> log likelihood: -1069494
INFO:lda:<130> log likelihood: -1069004
INFO:lda:<140> log likelihood: -1068914
INFO:lda:<150> log likelihood: -1068419
INFO:lda:<160> log likelihood: -1067596
INFO:lda:<170> log likelihood: -1067665
INFO:lda:<180> log likelihood: -1067560
INFO:lda:<190> log likelihood: -1067314
INFO:lda:<200> log likelihood: -1067031
INFO:lda:<210> log likelihood: -1066535
INFO:lda:<2

In [11]:
topic_word = model.topic_word_
doc_topic = model.doc_topic_
doc_topic = pd.DataFrame(doc_topic)
reviews_df = df1.reset_index().join(doc_topic)
music = pd.DataFrame()

for i in range(int(ntopics)):
    topic="topic_"+str(i)
    music[topic]=reviews_df.groupby('index')[i].mean()

music = music.reset_index()
topics = pd.DataFrame(topic_word)
topics.columns = cv.get_feature_names()
topics = topics.T

#Export the words/topics and documents/topics files

print ("Topics word distribution written in file topic_word_dist.xlsx ")
topics.to_excel("topic_word_dist.xlsx")
print ("Document topic distribution written in file document_topic_dist.xlsx ")
music.to_excel("document_topic_dist.xlsx",index=False)

Topics word distribution written in file topic_word_dist.xlsx 
Document topic distribution written in file document_topic_dist.xlsx 


## Now we are satisfied with the loading on each topic, let's look at what each topic represents. 

### Topic 1 - content
We can see that the first topic is related to music content, including artists, playlists, etc. 
- Playlists
- Features
- Content

In [12]:
topics.sort_values(by = 0, ascending = False)[:10]

,0,1,2,3
playlist,0.031078,4.143604e-07,2.939250e-02,4.345152e-03
artist,0.028639,4.143604e-07,2.096455e-07,2.078313e-03
album,0.021369,4.143604e-07,1.383870e-03,3.778064e-07
new,0.011929,2.279397e-03,5.492920e-03,3.778064e-07
feature,0.011026,4.143604e-07,2.641742e-03,3.778064e-07
love,0.010392,4.143604e-07,6.331502e-03,1.658608e-02
make,0.009660,4.143604e-07,5.618708e-03,1.261911e-02
great,0.009441,2.320833e-03,4.633374e-03,7.178700e-03
add,0.009148,4.143604e-07,6.666935e-03,6.763113e-03
way,0.008880,1.409240e-03,4.780126e-03,5.062984e-03


### Topic 2 - service/streaming
- Service 
- Streaming
- Quality

In [13]:
topics.sort_values(by = 1, ascending = False)[:10]

,0,1,2,3
tidal,3.805718e-03,0.023536,2.096455e-07,3.778064e-07
apple,5.244968e-03,0.021381,2.662707e-03,3.778064e-07
service,3.464201e-03,0.021008,2.096455e-07,3.778064e-07
quality,3.122684e-03,0.012058,2.096455e-07,3.778064e-07
streaming,3.537383e-03,0.010525,2.096455e-07,3.778064e-07
subscription,2.439406e-07,0.010442,2.727487e-04,2.493900e-03
ive,2.854349e-03,0.009945,1.115335e-02,6.989797e-03
time,5.684061e-03,0.009075,1.798779e-02,7.367603e-03
use,3.903294e-03,0.008495,7.191049e-03,5.705255e-03
sound,2.439650e-03,0.008288,2.096455e-07,3.665100e-03


### Topic 3 - issues/problems 
- Issues
- Problems
- Fixes

In [14]:
topics.sort_values(by = 2, ascending = False)[:10]

,0,1,2,3
playlist,3.107828e-02,4.143604e-07,0.029393,4.345152e-03
time,5.684061e-03,9.074907e-03,0.017988,7.367603e-03
playing,1.122371e-03,1.202060e-03,0.013313,6.804294e-04
want,8.391802e-03,4.143604e-07,0.012181,1.375253e-02
fix,2.439406e-07,2.237961e-03,0.011845,3.778064e-07
update,2.756773e-03,4.143604e-07,0.011405,3.778064e-07
issue,2.439406e-07,7.127413e-03,0.011342,3.778064e-07
ive,2.854349e-03,9.945064e-03,0.011153,6.989797e-03
problem,2.439406e-07,2.693757e-03,0.010734,4.534055e-03
phone,2.439406e-07,6.961669e-03,0.010210,2.229436e-03


### Topic 4 - ads/premium
- Ads
- Free
- Premium

In [15]:
topics.sort_values(by = 3, ascending = False)[:10]

,0,1,2,3
ad,2.439406e-07,4.143604e-07,2.096455e-07,0.031056
love,1.039212e-02,4.143604e-07,6.331502e-03,0.016586
soundcloud,2.146922e-03,4.143604e-07,3.061033e-03,0.015755
free,2.439406e-07,3.439606e-03,2.096455e-07,0.014886
really,5.220574e-03,4.143604e-07,8.763390e-03,0.014508
premium,2.439406e-07,5.805189e-04,1.216153e-03,0.014281
want,8.391802e-03,4.143604e-07,1.218061e-02,0.013753
make,9.660293e-03,4.143604e-07,5.618708e-03,0.012619
pay,7.808540e-04,4.682687e-03,2.096455e-07,0.012506
people,1.171159e-03,4.143604e-07,8.597560e-04,0.012241


## VADER sentiment analysis
We decided to use only the positive score in our final analysis.

In [17]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

In [18]:
negs = []
neus = []
poss = []
comp = []
df2 = df1.copy()
for i in range(len(df2)):
    review = df2.loc[i,'review']
    sentiment_dict = vader.polarity_scores(review)

    #neg = sentiment_dict['neg']
    #neu = sentiment_dict['neu']
    pos = sentiment_dict['pos']
    #compound = sentiment_dict['compound']

    #negs.append(neg)
    #neus.append(neu)
    poss.append(pos)
    #comp.append(compound)

#df2['compound'] = pd.Series(comp)
#df2['negative'] = pd.Series(negs)
#df2['neutral'] = pd.Series(neus)
df2['positive'] = pd.Series(poss)

df2

,userName,appName,rating,title,review,positive
0,V.143,Spotify,5,Best App,this app is by far the best music service ive ...,0.233
1,intense ligma,Spotify,5,Beat app but...,i personally love this app but the only part i...,0.150
2,GoldenShadow127,Spotify,3,It keeps stopping,a problem i frequently have is that when liste...,0.089
3,strwvrryle,Spotify,4,"bluetooth/wired connection to car, library format",i always connect to my scion ia via bluetooth ...,0.056
4,DeezNutsForPresident,Spotify,3,Issues with playing music,before i start i wanted to say that i wouldve ...,0.137
...,...,...,...,...,...,...
2995,austiniodtx,Tidal,5,High quality streaming,ive been a tidal user for a couple of year now...,0.207
2996,NateShay71,Tidal,4,"Great streaming service, could be amazing though",tidal is a great high quality way to stream mu...,0.188
2997,No. Just no.,Tidal,2,From 4 to 2 stars…,even after the recent update im still receivin...,0.103
2998,knowledge360,Tidal,1,Terrible customer service,my tidal stopped working one day i couldnt log...,0.072


In [19]:
# feature list
features = ['playlist', 'feature', 'content',\
            'service', 'quality', 'streaming',\
            'issue','problem','fix',\
            'ad','free','premium']

In [ ]:
df3 = df2.copy()
    
df3['word'] = df3.review.apply(lambda x: word_tokenize(x))
idxs = set()
for idx,row in df3.iterrows():
    words = row['word']
    for f in features:
        if f in words:
            idxs.add(idx)
            
rows = list(idxs)
df4 = df3.iloc[rows][['appName','review','positive']]

## SpaCy Similarity between each feature in the feature list and each review

In [21]:
import en_core_web_md
nlp = en_core_web_md.load()

In [22]:
for f in features:
    col = 'similarity_' + f
    df4[col] = 0
df4 = df4.reset_index().drop(columns='index')

In [23]:
# this code may take more than 20 min

col = 'similarity_' + f
j = 3
for f in features:
    for i in range (len(df4)):
        doc1 = (nlp(f))
        doc2 = (nlp(df4.loc[i]['review']))
        df4.iloc[i,j] = doc1.similarity(doc2)
    j += 1

In [24]:
df5 = df4.drop(columns='review')
df5

,appName,positive,similarity_playlist,similarity_feature,similarity_content,similarity_service,similarity_quality,similarity_streaming,similarity_issue,similarity_problem,similarity_fix,similarity_ad,similarity_free,similarity_premium
0,Spotify,0.233,0.255335,0.470600,0.655352,0.430110,0.538081,0.522374,0.423482,0.593743,0.240531,0.158363,0.426702,0.385291
1,Spotify,0.150,0.199469,0.294756,0.561823,0.317565,0.429339,0.409406,0.412769,0.607263,0.317067,0.146909,0.337301,0.263352
2,Spotify,0.089,0.187715,0.312724,0.540885,0.356079,0.422376,0.450057,0.405134,0.583244,0.274620,0.102150,0.351930,0.264261
3,Spotify,0.056,0.214923,0.328514,0.569786,0.362293,0.443703,0.437256,0.385899,0.579026,0.314099,0.126216,0.383931,0.290563
4,Spotify,0.137,0.179669,0.266505,0.512378,0.302696,0.387093,0.418826,0.369392,0.566079,0.296710,0.116635,0.338533,0.243813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2514,Tidal,0.207,0.192925,0.496866,0.607453,0.446521,0.553476,0.522024,0.385155,0.533371,0.140272,0.086220,0.383000,0.374979
2515,Tidal,0.188,0.190689,0.467266,0.650284,0.437490,0.568656,0.498442,0.418630,0.601415,0.176347,0.097726,0.373236,0.357488
2516,Tidal,0.103,0.208782,0.403777,0.594687,0.461358,0.462648,0.514616,0.474891,0.587551,0.200275,0.142513,0.362022,0.320689
2517,Tidal,0.072,0.179813,0.315961,0.561465,0.430506,0.454200,0.438029,0.426631,0.617319,0.286065,0.100139,0.379080,0.296591


## Total evaluation score for each review (we used VADER * Similarity_score)

In [94]:
df6 = df5[['appName','positive']]
for sim in df5.columns.to_list()[2:]:
    total = 'total_score_' + sim.split('_')[1]
    df6[total] = df6['positive'] * df5[sim]
    
df6 = df6.drop(columns='positive')
df6

,appName,total_score_playlist,total_score_feature,total_score_content,total_score_service,total_score_quality,total_score_streaming,total_score_issue,total_score_problem,total_score_fix,total_score_ad,total_score_free,total_score_premium
0,Spotify,0.059493,0.109650,0.152697,0.100216,0.125373,0.121713,0.098671,0.138342,0.056044,0.036899,0.099422,0.089773
1,Spotify,0.029920,0.044213,0.084273,0.047635,0.064401,0.061411,0.061915,0.091089,0.047560,0.022036,0.050595,0.039503
2,Spotify,0.016707,0.027832,0.048139,0.031691,0.037591,0.040055,0.036057,0.051909,0.024441,0.009091,0.031322,0.023519
3,Spotify,0.012036,0.018397,0.031908,0.020288,0.024847,0.024486,0.021610,0.032425,0.017590,0.007068,0.021500,0.016272
4,Spotify,0.024615,0.036511,0.070196,0.041469,0.053032,0.057379,0.050607,0.077553,0.040649,0.015979,0.046379,0.033402
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2514,Tidal,0.039935,0.102851,0.125743,0.092430,0.114570,0.108059,0.079727,0.110408,0.029036,0.017848,0.079281,0.077621
2515,Tidal,0.035850,0.087846,0.122253,0.082248,0.106907,0.093707,0.078702,0.113066,0.033153,0.018373,0.070168,0.067208
2516,Tidal,0.021505,0.041589,0.061253,0.047520,0.047653,0.053005,0.048914,0.060518,0.020628,0.014679,0.037288,0.033031
2517,Tidal,0.012947,0.022749,0.040425,0.030996,0.032702,0.031538,0.030717,0.044447,0.020597,0.007210,0.027294,0.021355


### Find the mean evaluation score for each music platform

In [95]:
mean_total = df6.groupby('appName').mean()
mean_total

,total_score_playlist,total_score_feature,total_score_content,total_score_service,total_score_quality,total_score_streaming,total_score_issue,total_score_problem,total_score_fix,total_score_ad,total_score_free,total_score_premium
appName,,,,,,,,,,,,
Amazon Music,0.029672,0.055864,0.083558,0.057059,0.067188,0.067021,0.058620,0.081589,0.033295,0.016441,0.052322,0.044446
Apple Music,0.029916,0.053440,0.080672,0.053851,0.063967,0.064691,0.056491,0.078554,0.033354,0.016642,0.049992,0.042268
SoundCloud,0.031767,0.056101,0.086783,0.057826,0.069006,0.069277,0.060783,0.085393,0.037503,0.020242,0.054627,0.045852
Spotify,0.030456,0.052022,0.083416,0.054561,0.065436,0.065629,0.058799,0.083019,0.037623,0.019888,0.052765,0.043378
Tidal,0.031896,0.062879,0.091012,0.063681,0.075098,0.073698,0.062765,0.086751,0.034085,0.016413,0.056700,0.049503
YouTube Music,0.030752,0.053565,0.082419,0.055097,0.065063,0.066280,0.057502,0.080469,0.034823,0.018081,0.051829,0.043490


In [96]:
lst = []

for score in mean_total.columns.to_list():
    df = pd.DataFrame(mean_total[score],index=mean_total.index).sort_values(score, ascending=False)
    lst.append(df)

## Rank each feature by app

In [97]:
lst

[               total_score_playlist
 appName                            
 Tidal                      0.031896
 SoundCloud                 0.031767
 YouTube Music              0.030752
 Spotify                    0.030456
 Apple Music                0.029916
 Amazon Music               0.029672,
                total_score_feature
 appName                           
 Tidal                     0.062879
 SoundCloud                0.056101
 Amazon Music              0.055864
 YouTube Music             0.053565
 Apple Music               0.053440
 Spotify                   0.052022,
                total_score_content
 appName                           
 Tidal                     0.091012
 SoundCloud                0.086783
 Amazon Music              0.083558
 Spotify                   0.083416
 YouTube Music             0.082419
 Apple Music               0.080672,
                total_score_service
 appName                           
 Tidal                     0.063681
 SoundCloud      

### Top platform for each feature

In [62]:
for df in lst:
    print(df.iloc[0])
    print()

total_score_playlist    0.031896
Name: Tidal, dtype: float64

total_score_feature    0.062879
Name: Tidal, dtype: float64

total_score_content    0.091012
Name: Tidal, dtype: float64

total_score_service    0.063681
Name: Tidal, dtype: float64

total_score_quality    0.075098
Name: Tidal, dtype: float64

total_score_streaming    0.073698
Name: Tidal, dtype: float64

total_score_issue    0.062765
Name: Tidal, dtype: float64

total_score_problem    0.086751
Name: Tidal, dtype: float64

total_score_fix    0.037623
Name: Spotify, dtype: float64

total_score_ad    0.020242
Name: SoundCloud, dtype: float64

total_score_free    0.0567
Name: Tidal, dtype: float64

total_score_premium    0.049503
Name: Tidal, dtype: float64



## Finding: Tidal dominates in almost all features